This notebook renders a few example images and videos.

Dependencies:

* OSM - based graph of germany at `h3_resolution`
* Running `route3_road` instance using that graph.
* `ffmpeg`

In [ ]:
import os.path
from pathlib import Path

notebookdir = Path(os.path.dirname(os.path.realpath("__file__")))

# additional libraries to the dev-dependencies of route3_road_client
%pip install rasterio==1.2.10 matplotlib==3.5.1 earthpy==0.9.4 h3ronpy

import os
import subprocess
import numpy as np
import matplotlib.pyplot as plt
import earthpy as et
import earthpy.spatial as es
import earthpy.plot as ep
import rasterio as rio
import geopandas as gpd
from rasterio.windows import transform
from rasterio.plot import plotting_extent
from h3ronpy.raster import raster_to_dataframe, nearest_h3_resolution
from h3ronpy.util import dataframe_to_geodataframe
import h3.api.numpy_int as h3

bounds = (8.0, 48.2, 8.42, 48.42)
h3_resolution = 10
linewidth=0.2

In [ ]:
#https://earthpy.readthedocs.io/en/latest/gallery_vignettes/plot_dem_hillshade.html
#with rio.open(os.path.expanduser("~/tmp/Copernicus_DSM_COG_10_N48_00_E008_00_DEM.tif")) as src:  

def fetch_elevation():
    with rio.Env(AWS_NO_SIGN_REQUEST="YES") as env:
        with rio.open("s3://copernicus-dem-30m/Copernicus_DSM_COG_10_N48_00_E008_00_DEM/Copernicus_DSM_COG_10_N48_00_E008_00_DEM.tif") as src:
            window = src.window(*bounds)
            window_transform = transform(window, src.transform)
            elevation = src.read(1, window=window)
            # Set masked values to np.nan
            elevation[elevation < 0] = np.nan
            
            pe = plotting_extent(elevation, transform=window_transform)
            return pe, elevation

def fetch_population():
    with rio.Env(AWS_NO_SIGN_REQUEST="YES") as env:
        with rio.open("s3://dataforgood-fb-data/hrsl-cogs/hrsl_general/hrsl_general-latest.vrt") as src:
            window = src.window(*bounds)
            window_transform = transform(window, src.transform)
            population_band = src.read(1, window=window)
            axis_order = "yx"
            
            conversion_h3_res = nearest_h3_resolution(population_band.shape, window_transform, axis_order=axis_order)
            
            population_band[np.isnan(population_band)] = 0
            
            df = raster_to_dataframe(population_band, window_transform, conversion_h3_res,
                                                nodata_value=0,
                                                compacted=False, axis_order=axis_order)
            df.rename(columns = {'value':'num_people'}, inplace = True)
            if h3_resolution > conversion_h3_res:
                raise ValueError(f"only h3 resolutions up to {conversion_h3_res}")
            elif h3_resolution < conversion_h3_res:
                # scale down
                df["h3index"] = df["h3index"].apply(lambda i: h3.h3_to_parent(i, h3_resolution))
                return df.groupby(by=["h3index"]).sum().reset_index()
            else: 
                return df         

In [ ]:
# Create and plot the hillshade with earthpy
hillshade_plotting_extent, elevation_arr = fetch_elevation()
hillshade = es.hillshade(elevation_arr, azimuth=270)
population = dataframe_to_geodataframe(fetch_population())

In [ ]:
# print(hillshade_plotting_extent, bounds)

fig, ax = plt.subplots(figsize=(24, 24))

population.plot("num_people",
                ax=ax, 
                zorder=10, 
                alpha=0.9, 
                cmap="viridis", 
                edgecolor="black", 
                linewidth=linewidth,
                #legend=True,
)

p_hillshade = ep.plot_bands(
    hillshade,
    ax=ax,
    cbar=False,
    alpha=0.3,
    extent=hillshade_plotting_extent,
    #title="Hillshade made from DTM",
)

plt.show()

In [ ]:
import route3_road_client as rrc
import pyarrow as pa
from shapely.geometry import Polygon

def table_to_geodataframe(table: pa.Table, column_name: str = "h3index") -> gpd.GeoDataFrame:
    df = table.to_pandas()
    return gpd.GeoDataFrame(df,
                            geometry=[Polygon(h3.h3_to_geo_boundary(h, geo_json=True)) for h in
                                      np.nditer(df[column_name].to_numpy())],
                            crs=4326)


server = rrc.Server(hostport="127.0.0.1:7088")
graph_handle = rrc.build_graph_handle("germany", h3_resolution)

In [ ]:
def calc_upto_some_threshold(gh, num_secs):
    origins = [   
        # h3.geo_to_h3(48.28713017627725, 8.111112713813782, gh.h3_resolution), # Fischerbach
        h3.geo_to_h3(48.291916615643935, 8.218567371368408, gh.h3_resolution) # Wolfach
    ]
    
    request = rrc.build_h3_within_threshold_request(
        gh,
        origins,
        travel_duration_secs_threshold=int(num_secs)
    )
    return server.h3_cells_within_threshold(request).table

from mpl_toolkits.axes_grid1 import make_axes_locatable
import tempfile

def within_threshold_render_frames(max_secs, step_secs, figsize=(14,14), labelsize=12):
    outdir = Path(tempfile.gettempdir()) / "vframes"
    if outdir.exists():
        import shutil
        shutil.rmtree(outdir)
    os.mkdir(outdir)

    gdf = table_to_geodataframe(calc_upto_some_threshold(graph_handle, max_secs), column_name="h3index_cell_origin")

    for secs in range(step_secs, max_secs, step_secs):
        step_gdf = gdf[gdf["travel_duration_secs"] <= secs]
        fig, ax = plt.subplots(figsize=figsize)

        divider = make_axes_locatable(ax)
        cax = divider.append_axes("right", size="2%", pad=0.1)
        cax.tick_params(labelsize=labelsize)

        step_gdf.plot("travel_duration_secs",
                        ax=ax, 
                        zorder=10, 
                        alpha=0.9, 
                        cmap="viridis", 
                        edgecolor="black", 
                        linewidth=linewidth,
                        legend=True,
                        vmin=0, 
                        vmax=max_secs,
                        cax=cax,
        )

        p_hillshade = ep.plot_bands(
            hillshade,
            ax=ax,
            cbar=False,
            alpha=0.3,
            extent=hillshade_plotting_extent,
            title="Travel durations in the H3-grid graph [seconds]",
        )
        frame_num = int(secs/step_secs)
        outpng = outdir / f'f_{frame_num:05}.png'
        print(f"rendering {outpng}")
        plt.savefig(outpng, pad_inches=0, bbox_inches='tight')
        plt.close(fig)
    return outdir

#plt.show()

outdir = within_threshold_render_frames(60*20, 60*2)
# http://blog.pkh.me/p/21-high-quality-gif-with-ffmpeg.html
gif_filters="scale=800:-1:flags=lanczos"
gif_palette = outdir / "palette.png"
subprocess.check_call(["ffmpeg", "-y", "-i", f"{str(outdir)}/f_%05d.png", 
                      '-vf', f"{gif_filters},palettegen", str(gif_palette)])
subprocess.check_call(["ffmpeg", "-y", "-framerate", "2", "-i", f"{str(outdir)}/f_%05d.png", "-i", str(gif_palette), 
                       "-lavfi", f"{gif_filters} [x]; [x][1:v] paletteuse", 
                       f'{str(notebookdir)}/within-threshold.gif'])


outdir = within_threshold_render_frames(60*20, 5, labelsize=20, figsize=(24, 24))
subprocess.check_call(["ffmpeg", "-y", "-framerate", "15", "-i", f"{str(outdir)}/f_%05d.png", 
                       '-crf', '25',
                       '-b', '10M',
                       "-pix_fmt", "yuv420p",
                       "-c:v", "libx264", # to be supported in firefox
                       "-movflags", "+faststart", # browser-specific 
                       "-filter:v", "crop='floor(in_w/2)*2:floor(in_h/2)*2'", # yuv420p only supports even width and height
                       f'{str(notebookdir)}/within-threshold.mp4'])